In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import TwoSlopeNorm
import torch.nn as nn

from train_bot import ChessNet

# Load weights
model = ChessNet()
state = torch.load('best.pth', map_location='cpu')
model.load_state_dict(state)



<All keys matched successfully>

In [ ]:
np.vstack((
    model.fc1.weight.detach().cpu().numpy()[0][:384].reshape(6, 64),
    model.fc1.weight.detach().cpu().numpy()[0][384:].reshape(6, 64)
)).reshape(12, 8, 8)

array([[ 0.08006161,  0.08006161,  0.08006161,  0.08006161,  0.08006161,
         0.08006161,  0.08006161,  0.08006161,  0.08041947,  0.08003852,
         0.08245135,  0.08048303,  0.08112696,  0.08023699,  0.08255173,
         0.08236162,  0.08006161,  0.08006161,  0.08006161,  0.08006161,
         0.083299  ,  0.08329833,  0.08006161,  0.08006161,  0.08329794,
         0.08006161,  0.08329755,  0.08329906,  0.08006161,  0.08329395,
         0.0832812 ,  0.08006161,  0.08006161,  0.08006161,  0.08006161,
         0.08006161,  0.08006161,  0.11669952,  0.08006161,  0.08006161,
         0.08006161,  0.08006161,  0.08006161,  0.08006161,  0.08006161,
         0.08006161,  0.08006161,  0.08006161,  0.08006161,  0.08006161,
         0.08006161,  0.08006161,  0.08006161,  0.08006161,  0.08006161,
         0.08006161,  0.08006161,  0.08006161,  0.08006161,  0.08006161,
         0.08006161,  0.08006161,  0.08006161,  0.08006161],
       [ 0.28001702,  0.28254062,  0.28001702,  0.28001702,  0.

In [10]:
# Extract and reshape
weights = model.fc.weight.detach().cpu().numpy().reshape(12, 64)
boards = weights.reshape(12, 8, 8)

# Colormap norm around zero
max_abs = np.max(np.abs(boards))
norm = TwoSlopeNorm(vmin=-max_abs, vcenter=0, vmax=max_abs)

names = [
    'White Pawn', 'White Knight', 'White Bishop', 'White Rook',
    'White Queen', 'White King',
    'Black Pawn', 'Black Knight', 'Black Bishop', 'Black Rook',
    'Black Queen', 'Black King'
]

files = list('abcdefgh')
ranks = list('12345678')

for board, name in zip(boards, names):
    fig, ax = plt.subplots(figsize=(4, 4))
    im = ax.imshow(board, cmap='RdYlGn', norm=norm, origin='lower')
    ax.set_xticks(range(8))
    ax.set_xticklabels(files)
    ax.set_yticks(range(8))
    ax.set_yticklabels(ranks)
    ax.set_xlabel('File')
    ax.set_ylabel('Rank')
    ax.set_title(name)
    fig.colorbar(im, ax=ax, shrink=0.6, orientation='vertical', label='Weight')
    plt.tight_layout()
    plt.show()


AttributeError: 'ChessNet' object has no attribute 'fc'